### Используемые сторонние библотеки

* keras (2.4.3)
* matplotlib (3.2.1)
* tensorflow (2.2.0)
* pandas (1.0.3)
* sklearn (0.22.1)
* numpy (1.18.1)

In [1]:
# игнорируем warnings, чтобы не захламлять вывод
import warnings
warnings.filterwarnings("ignore")


In [2]:
# импортируем зависимости
import pandas as pd
from pandas import DataFrame
import tensorflow as tf
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Conv1D, Dropout, Dense
import keras
import matplotlib.pyplot as plt
from math import sqrt
plt.style.use('ggplot')
from numpy import array
import pandas as pd
import numpy as np



In [3]:
# функции, связанные с предобработкой

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Создает массив возможных кусков из временного ряда, 
    таких что они включают n_in элементов как обучающую часть, 
    и n_out как предсказываемую. По сути устраняет "краевые эфекты"
    Метод взят из интернета.
    """
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def difference(dataset, interval=1):
    
    """
    Считает разницу между элементами временного ряда,
    отстоящими друг от друга на interval
    """
    
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

def prepare_data(series, n_test, n_lag, n_seq):
    
    """
    Преобразует данные с помощью нахождения разности (тк данные имеют четкий тренд) и шкалирования,
    формирует тренировочную и обучающую части выборки
    """
    
    raw_values = series.values
    
    diff_series = difference(raw_values, 1)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)

    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(diff_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values
    
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test

In [4]:
# функция построения сети

def fit_lstm(
    train, 
    n_lag, 
    n_seq, 
    nb_epoch, 
    n_neurons, 
    learning_rate, 
    validation_size, 
    n_batch=1,
    verbose=2
):
    
    """
    Создает и обучает нейросеть с учетом выбранных параметров.
    Возвращает саму модель и историю изменения метрик loss и val_loss в процессе обучения.
    """
    
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    
    
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    
    history = []
    
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    
    #model.add(Conv1D(32, kernel_size=3))
    
    model.add(LSTM(
        n_neurons, 
        batch_input_shape=(n_batch, X.shape[1], X.shape[2]), 
        stateful=True))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='tanh'))
    
    
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy', 'mae'])
    
        
    history = model.fit(
        X, y, 
        epochs=n_epochs, 
        batch_size=n_batch, 
        validation_split=validation_size, 
        verbose=verbose, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=16, min_delta=1e-4,restore_best_weights=True)]
    )

    return model, history.history

In [5]:
# функции, выполняющие прогноз

def forecast_lstm(model, X, n_batch):
    
    """
    Предсказание предстоящих значений после заданной точки
    """
    
    X = X.reshape(1, 1, len(X))
    forecast = model.predict(X, batch_size=n_batch)
    return [x for x in forecast[0, :]]


def make_forecasts(model, test, n_lag, n_seq, n_batch=1):
    """
    Делает предсказания с помощью переданной модели
    для всех временных участков из test
    """
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        forecast = forecast_lstm(model, X, n_batch)
        forecasts.append(forecast)
    return forecasts


def inverse_difference(last_ob, forecast):
    """
    Перевод значений из разности к абсолютным 
    """
    
    inverted = list()
    inverted.append(forecast[0] + last_ob)
    
    for i in range(1, len(forecast)):
        inverted.append(forecast[i] + inverted[i-1])
    return inverted


def inverse_transform(series, forecasts, scaler, n_test):
    
    """
    Обратное преобразование данных (после шкалирования и взятия разности)
    """
    
    inverted = list()
    for i in range(len(forecasts)):
        
        forecast = array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        
        index = len(series) - n_test + i - 1
        last_ob = series.values[index]
        inv_diff = inverse_difference(last_ob, inv_scale)
        
        inverted.append(inv_diff)
    return inverted

In [12]:
# функции для численной и визуальной оценки качества модели
from sklearn.metrics import accuracy_score

def evaluate_forecasts_acc(test, forecasts, n_lag, n_seq):
    """
    Расчет точности по направлению изменения
    """
    actual = [row for row in test]
    predicted = [forecast for forecast in forecasts]
    
    actual_binary = []
    predicted_binary = []
    
    for j in range(1,len(predicted)):
        actual_diff = np.sign(np.diff(
            [actual[j-1][0]] + actual[j]
        ))
        pred_diff = np.sign(np.diff(
            [predicted[j-1][0]] + predicted[j]
        ))
        
        actual_binary += list(actual_diff)
        predicted_binary += list(pred_diff)
        
    print('Binary acccuracy:', accuracy_score(actual_binary, predicted_binary))


def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    
    """
    Расчет и вывод метрики RMSE для каждой удаленности предсказания
    """
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
                  
        
        
        rmse = sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))

def plot_forecasts(series, c, forecasts, n_test, save=None):
    
    """
    Отрисовка графика предсказаний из каждой точки тестовой выборки
    График полный (Включает весь train sample).
    """
    
    plt.figure(figsize=(14,7))
    
    plt.plot(series.values)
    
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series.values[off_s]] + forecasts[i]
        plt.plot(xaxis, yaxis, color='green')
        
    
    plt.title('Forecasts Plot: ' + c)
    plt.xlabel('Time point')
    plt.ylabel('log value')
    if save is not None:
        plt.tight_layout()
        plt.savefig(save, dpi=300)
        
    plt.close()
    
def plot_forecasts_cropped(series, c, forecasts, n_test, save=None):
    
    """
    Отрисовка графика предсказаний из каждой точки тестовой выборки
    График обрезанный для лучшей читаемости
    (Включает небольшую часть train sample).
    """
    
    
    plt.figure(figsize=(14,7))
    
    plt.plot(series.values)
    
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series.values[off_s]] + forecasts[i]
        plt.plot(xaxis, yaxis, color='green')
        
    plt.xlim([len(series) - 2*n_test, len(series)])
    plt.ylim(
        [min(series[-2*n_test:]), max(series[-2*n_test:])]
    )
    
    plt.title('Forecasts Plot: ' + c)
    plt.xlabel('Time point')
    plt.ylabel('log value')
    if save is not None:
        plt.tight_layout()
        plt.savefig(save, dpi=300)
    plt.close()

In [14]:
# В ЭТОЙ ЯЧЕЙКЕ ЦИКЛ ДЛЯ ИМПОРТА! ДЛЯ ЭКСПОРТА В СЛЕДУЮЩЕЙ ЯЧЕЙКЕ!


# Все параметры

n_lag = 36 # длина фрагмента временного ряда, используемая для предсказания
n_seq = 3 # количество временных точек, для которых будет строиться предсказание
n_test = 100 # абсолютный размер тестовой выборки
validation_size=0.1 # относительный размер валидационной выборки из трейна
n_epochs = 1000 # число эпох обучения
n_neurons = 64 # число LSTM нейронов
learning_rate = 1e-5 # как быстро двигаемся по весам в процессе оптимизации
LOG = True # будем ли логарифимировать


all_countries = [
    'Russian Federation',
    'United States',
    'United Kingdom',
    'Germany',
    'China, Hong Kong SAR',
    'Canada',
    'Japan',
    'Turkey',
    'Denmark',
    'Maldives',
    'Sweden',
    'Korea, Republic of',
    'Switzerland',
    'Tunisia',
    'Malaysia',
    'Argentina',
]

# читаем табличку, вытаскиваем данные импорта США как удобный вариант для обучения
dataframe = pd.read_csv('MBSComtrade.csv')
dataframe['period_in_date'] = pd.to_datetime(dataframe['period_in_date'])

print('IMPORT RESULTS')
print(
    'Country',
    'val_loss',
    'val_accuracy',
    'val_mae',
    sep='\t'
)


for country in all_countries:

    test_c = country
    country_subset = dataframe[dataframe.country_english_name == test_c]

    country_import = country_subset[country_subset.trade_flow_desc == 'Imports']
    country_export = country_subset[country_subset.trade_flow_desc == 'Exports']

    series = country_import.value


    # если надо, логарифмируем
    if LOG == True:

        for i in series.index:
            series[i] = np.log(series[i])


    # готовим данные
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)

    # строим модель
    model, history = fit_lstm(
        train, 
        n_lag, 
        n_seq, 
        n_epochs, 
        n_neurons, 
        learning_rate,
        validation_size,
        verbose=0,
    )
    
    print(
        country, 
        round(history['val_loss'][-1], 4), 
        round(history['val_accuracy'][-1], 4), 
        round(history['val_mae'][-1], 4), 
        sep='\t'
    )

    #рисуем метрики
    fig, axs = plt.subplots(1,2, figsize=(14,7))

    axs[0].plot(history['val_loss'])
    axs[0].set_title('Validation loss: '+ test_c)
    axs[0].set_xlabel('Step')
    axs[0].set_ylabel('loss')


    axs[1].plot(history['val_mae'])
    axs[1].set_title('Validation MAE: ' + test_c)
    axs[1].set_xlabel('Step')
    axs[1].set_ylabel('MAE')


    plt.tight_layout()
    plt.savefig('Models_quality_pics_import/ModelQual_{}.png'.format(test_c), dpi=300)
    plt.close()



    # строим предсказания
    forecasts = make_forecasts(model, test, n_lag, n_seq)
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)


    # вытаскиваем реальные сценарии
    actual = [row[n_lag:] for row in test]
    actual = inverse_transform(series, actual, scaler, n_test+2)

    # на них считаем метрики RMSE
    evaluate_forecasts_acc(actual, forecasts, n_lag, n_seq)

    # рисуем
    plot_forecasts(
        series, 
        test_c,
        forecasts, 
        n_test+2, 
        save='Forecasts_pics_import/Forecasts_{}.png'.format(test_c)
    )
    
    plot_forecasts_cropped(
        series, 
        test_c,
        forecasts, 
        n_test+2, 
        save='Forecasts_pics_import/Forecasts_crop_{}.png'.format(test_c)
    )

IMPORT RESULTS
Country	val_loss	val_accuracy	val_mae
Russian Federation	0.3106	0.4286	0.4399
Binary acccuracy: 0.6868686868686869
United States	0.4077	0.6984	0.4812
Binary acccuracy: 0.569023569023569
United Kingdom	0.3197	0.3651	0.4157
Binary acccuracy: 0.4882154882154882
Germany	0.5432	0.6222	0.5841
Binary acccuracy: 0.6902356902356902
China, Hong Kong SAR	0.4966	0.4603	0.5391
Binary acccuracy: 0.5925925925925926
Canada	0.3112	0.6825	0.411
Binary acccuracy: 0.6902356902356902
Japan	0.5868	0.7302	0.5511
Binary acccuracy: 0.7104377104377104
Turkey	0.19	0.5556	0.3308
Binary acccuracy: 0.5892255892255892
Denmark	0.377	0.5873	0.4878
Binary acccuracy: 0.6734006734006734
Maldives	0.1179	0.4286	0.2897
Binary acccuracy: 0.48148148148148145
Sweden	0.4075	0.6667	0.5022
Binary acccuracy: 0.6296296296296297
Korea, Republic of	0.1987	0.5079	0.3252
Binary acccuracy: 0.5387205387205387
Switzerland	0.3455	0.6667	0.4511
Binary acccuracy: 0.7171717171717171
Tunisia	0.365	0.4138	0.4575
Binary acccuracy:

In [15]:
# В ЭТОЙ ЯЧЕЙКЕ ЦИКЛ ДЛЯ ЭКСПОРТА! ДЛЯ ИМПОРТА В ПРЕДЫДУЩЕЙ ЯЧЕЙКЕ!


# Все параметры

n_lag = 36 # длина фрагмента временного ряда, используемая для предсказания
n_seq = 3 # количество временных точек, для которых будет строиться предсказание
n_test = 100 # абсолютный размер тестовой выборки
validation_size=0.1 # относительный размер валидационной выборки из трейна
n_epochs = 1000 # число эпох обучения
n_neurons = 64 # число LSTM нейронов
learning_rate = 1e-5 # как быстро двигаемся по весам в процессе оптимизации
LOG = True # будем ли логарифимировать


all_countries = [
    'Russian Federation',
    'United States',
    'United Kingdom',
    'Germany',
    'China, Hong Kong SAR',
    'Canada',
    'Japan',
    'Turkey',
    'Denmark',
    'Maldives',
    'Sweden',
    'Korea, Republic of',
    'Switzerland',
    'Tunisia',
    'Malaysia',
    'Argentina',
]

# читаем табличку, вытаскиваем данные импорта США как удобный вариант для обучения
dataframe = pd.read_csv('MBSComtrade.csv')
dataframe['period_in_date'] = pd.to_datetime(dataframe['period_in_date'])


print('EXPORT RESULTS')
print(
    'Country',
    'val_loss',
    'val_accuracy',
    'val_mae',
    sep='\t'
)

for country in all_countries:

    test_c = country
    country_subset = dataframe[dataframe.country_english_name == test_c]

    country_import = country_subset[country_subset.trade_flow_desc == 'Imports']
    country_export = country_subset[country_subset.trade_flow_desc == 'Exports']

    series = country_export.value


    # если надо, логарифмируем
    if LOG == True:

        for i in series.index:
            series[i] = np.log(series[i])


    # готовим данные
    scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)

    # строим модель
    model, history = fit_lstm(
        train, 
        n_lag, 
        n_seq, 
        n_epochs, 
        n_neurons, 
        learning_rate,
        validation_size,
        verbose=0,
    )
    
    print(
        country, 
        round(history['val_loss'][-1], 4), 
        round(history['val_accuracy'][-1], 4), 
        round(history['val_mae'][-1], 4), 
        sep='\t'
    )

    #рисуем метрики
    fig, axs = plt.subplots(1,2, figsize=(14,7))

    axs[0].plot(history['val_loss'])
    axs[0].set_title('Validation loss: '+ test_c)
    axs[0].set_xlabel('Step')
    axs[0].set_ylabel('loss')


    axs[1].plot(history['val_mae'])
    axs[1].set_title('Validation MAE: ' + test_c)
    axs[1].set_xlabel('Step')
    axs[1].set_ylabel('MAE')


    plt.tight_layout()
    plt.savefig('Models_quality_pics_export//ModelQual_{}.png'.format(test_c), dpi=300)
    plt.close()



    # строим предсказания
    forecasts = make_forecasts(model, test, n_lag, n_seq)
    forecasts = inverse_transform(series, forecasts, scaler, n_test+2)


    # вытаскиваем реальные сценарии
    actual = [row[n_lag:] for row in test]
    actual = inverse_transform(series, actual, scaler, n_test+2)

    # на них считаем метрики RMSE
    evaluate_forecasts_acc(actual, forecasts, n_lag, n_seq)

    # рисуем
    plot_forecasts(
        series, 
        test_c,
        forecasts, 
        n_test+2, 
        save='Forecasts_pics_export/Forecasts_{}.png'.format(test_c)
    )
    
    plot_forecasts_cropped(
        series, 
        test_c,
        forecasts, 
        n_test+2, 
        save='Forecasts_pics_export/Forecasts_crop_{}.png'.format(test_c)
    )

EXPORT RESULTS
Country	val_loss	val_accuracy	val_mae
Russian Federation	0.4719	0.2857	0.4975
Binary acccuracy: 0.5555555555555556
United States	0.2308	0.6984	0.371
Binary acccuracy: 0.5959595959595959
United Kingdom	0.3426	0.3016	0.4241
Binary acccuracy: 0.468013468013468
Germany	0.5252	0.7111	0.5816
Binary acccuracy: 0.7373737373737373
China, Hong Kong SAR	0.3401	0.6032	0.4442
Binary acccuracy: 0.6127946127946128
Canada	0.5554	0.5714	0.5467
Binary acccuracy: 0.5521885521885522
Japan	0.1774	0.7778	0.3233
Binary acccuracy: 0.8585858585858586
Turkey	0.2865	0.3333	0.4425
Binary acccuracy: 0.5892255892255892
Denmark	0.7538	0.5397	0.7162
Binary acccuracy: 0.6195286195286195
Maldives	0.5778	0.3333	0.617
Binary acccuracy: 0.46464646464646464
Sweden	0.4067	0.6667	0.51
Binary acccuracy: 0.6127946127946128
Korea, Republic of	0.178	0.4762	0.3147
Binary acccuracy: 0.5420875420875421
Switzerland	0.2523	0.6825	0.38
Binary acccuracy: 0.7138047138047138
Tunisia	0.1787	0.2586	0.3472
Binary acccuracy: 0